# Decision trees

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#The-problem" data-toc-modified-id="The-problem-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>The problem</a></span><ul class="toc-item"><li><span><a href="#Paréntesis:-build-a-linear-regression" data-toc-modified-id="Paréntesis:-build-a-linear-regression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Paréntesis: build a linear regression</a></span></li><li><span><a href="#Back-to-trees" data-toc-modified-id="Back-to-trees-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Back to trees</a></span></li></ul></li><li><span><a href="#Data-exploration" data-toc-modified-id="Data-exploration-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data exploration</a></span></li><li><span><a href="#Train-test-splitting" data-toc-modified-id="Train-test-splitting-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Train test splitting</a></span></li><li><span><a href="#Models" data-toc-modified-id="Models-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Models</a></span><ul class="toc-item"><li><span><a href="#Baseline-model" data-toc-modified-id="Baseline-model-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Baseline model</a></span></li><li><span><a href="#Simple-tree-(depth=1)" data-toc-modified-id="Simple-tree-(depth=1)-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Simple tree (depth=1)</a></span></li><li><span><a href="#Bigger-tree-(depth=3)" data-toc-modified-id="Bigger-tree-(depth=3)-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Bigger tree (depth=3)</a></span></li><li><span><a href="#Huge-tree-(depth=20)" data-toc-modified-id="Huge-tree-(depth=20)-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Huge tree (depth=20)</a></span></li><li><span><a href="#Overfitting" data-toc-modified-id="Overfitting-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Overfitting</a></span></li><li><span><a href="#Other-hyperparameters" data-toc-modified-id="Other-hyperparameters-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>Other hyperparameters</a></span></li><li><span><a href="#Grid-search" data-toc-modified-id="Grid-search-5.7"><span class="toc-item-num">5.7&nbsp;&nbsp;</span>Grid search</a></span></li></ul></li><li><span><a href="#Feature-importance" data-toc-modified-id="Feature-importance-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Feature importance</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Summary</a></span></li></ul></div>

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

## Intro

A decision tree tries to predict the target variable using a logic like the following.

<img src="https://lh4.googleusercontent.com/v9UQUwaQTAXVH90b-Ugyw2_61_uErfYvTBtG-RNRNB_eHUFq9AmAN_2IOdfOETnbXImnQVN-wPC7_YzDgf7urCeyhyx5UZmuSwV8BVsV8VnHxl1KtgpuxDifJ4pLE23ooYXLlnc" width=600>

Decision trees:
 * are used **both** for classification (previous example Fit/Unfit) and regression
 * involve stratifying (segmenting) the predictor space...
 * in an iterative manner
 * are given this name because splitting rules can be summarized in a tree

Decision trees:
 * are simple
 * are useful for interpretation
 * are not very powerful predictors but...
 * give rise to more complex models, like Random Forest or Gradient Boosted Trees algorithms

## The problem

Today we will be using a **white wine** dataset

Experts have rated several wines, whose physical properties are also given

In [ ]:
df = pd.read_csv("../datasets/wine_quality.csv")

In [ ]:
df.shape

In [ ]:
df.sample(10)

We want to:
 * build a **supervised** learning model
 * which is a **regression** model (predict quantitative feature)
 * that tries to predict wine `quality` from its physical properties (so that we do not anymore need experts' advice)

### Paréntesis: build a linear regression

1. Compro en la tienda una linear regression.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lin = LinearRegression()

In [ ]:
# train test split
from sklearn.model_selection import train_test_split

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
X = df.drop("quality", axis=1)

In [ ]:
y = df.quality

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=66)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

2. La entreno con el chorro de 4000 vinos de train. Qué ha cambiado.

In [ ]:
lin.fit(X_train, y_train)
# encontrar los parámetros beta_0, ... beta_n

In [ ]:
pd.Series(lin.coef_, index=X.columns)

3. Cojo un vino nuevo. Cómo se predice su quality?  
La prediccion de un vino nuevo será beta_0 + beta_1 * fixed_acidity + beta_2 * volatile... + ...

In [ ]:
lin.predict(X_test[:5]).round(2)

In [ ]:
y_test[:5]

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_train, lin.predict(X_train))

In [ ]:
mean_squared_error(y_test, lin.predict(X_test))

Error on test is usually higher

### Back to trees

We will do train test splitting for correct asessment of model performance

We will use MSE metric: $$MSE=\frac{1}{N}\sum(\hat{y} - y)^2$$

In [ ]:
from sklearn.metrics import mean_squared_error

We will:
 * try several models and...
 * keep the one with the **least** MSE on **test set** (also called test error)
 * anyways, we will always show training error too

## Data exploration

In [ ]:
df.head()

In [ ]:
df.isna().sum().sum()

In [ ]:
df.columns

In [ ]:
df.columns = [col.replace(" ", "_") for col in df.columns]

In [ ]:
df.columns

In [ ]:
sns.countplot(x=df.quality, palette="Blues")

## Train test splitting

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
target = "quality"

In [ ]:
# predictors
X = df.drop(target, axis=1)
# target
y = df[target]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=666)

In [ ]:
X.shape

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

## Models

### Baseline model

The baseline model predicts the mean quality for every wine

In [ ]:
baseline = y_train.mean()

In [ ]:
baseline

MSE can be manually computed

Train error

In [ ]:
((y_train - baseline) ** 2).mean()

Test error

In [ ]:
((y_test - baseline) ** 2).mean()

Test error is, indeed, as usual, bigger than train error

### Simple tree (depth=1)

Lets first fit a Tree, then interpret it

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model = DecisionTreeRegressor(max_depth=1)

In [ ]:
model.fit(X_train, y_train)

Lets see how this model predicts the first 5 wines

In [ ]:
# real values
y_train[:5]

In [ ]:
# predicted values
model.predict(X_train[:5].values).round(2)

Train error

In [ ]:
mean_squared_error(
    y_true=y_train,
    y_pred=model.predict(X_train)
)

Test error

In [ ]:
mean_squared_error(
    y_true=y_test,
    y_pred=model.predict(X_test)
)

In [ ]:
from sklearn.tree import plot_tree

In [ ]:
df.head()

In [ ]:
y_train.mean()

In [ ]:
fig = plt.figure(figsize=(10, 6))
plot_tree(model, feature_names=df.columns[:-1], filled=True);

In [ ]:
mean_squared_error(y_train, model.predict(X_train))

In [ ]:
model.predict(X_test[5:10].values).round(2)

Some important questions for deep understanding:
 1. while training, why did the DecisionTree choose `alcohol` and $10.85$ ?

In [ ]:
df.head()

I will do the intestines of decision tree for pair (feature, cut) of (residual_sugar, 5)

Imagine we chose `residual_sugar` and value 5

In [ ]:
group1 = X_train[X_train.residual_sugar <= 5].copy()
group2 = X_train[X_train.residual_sugar > 5].copy()

In [ ]:
group1.shape

In [ ]:
group2.shape

In [ ]:
group1_mean = y_train[group1.index].mean()

In [ ]:
group1_mean

In [ ]:
group2_mean = y_train[group2.index].mean()

In [ ]:
group2_mean

In [ ]:
mse = (
    ((y_train[group1.index] - group1_mean) ** 2).sum() +
    ((y_train[group2.index] - group2_mean) ** 2).sum()
) / X_train.shape[0]

In [ ]:
mse

Shitty improvement over baseline model, and much worse than alcohol 10.85, optimal feature-threshold pair

2. what is the meaning of `mse`: the mean squared error in the bucket: the one you would get if every wine in that bucket was given the mean of the bucket. This is, a baseline model in the bucket.
3. what is the meaning of `value`: the mean quality of wines in that bucket. The value that will be predicted for every new wine ending in that tree leaf

4. while testing (predicting a new instance), how does the tree operate? It goes through a unique path. When this path ends, the value in that leaf is the prediction

### Bigger tree (depth=3)

In [ ]:
model = DecisionTreeRegressor(max_depth=3)

In [ ]:
%%time
model.fit(X_train, y_train)

In [ ]:
# real values
y_train[:5]

In [ ]:
# predicted values
model.predict(X_train[:5].values).round(2)

Train error

In [ ]:
mean_squared_error(
    y_true=y_train,
    y_pred=model.predict(X_train)
)

Test error

In [ ]:
mean_squared_error(
    y_true=y_test,
    y_pred=model.predict(X_test)
)

In [ ]:
fig = plt.figure(figsize=(25,20))
plot_tree(model, feature_names=df.columns[:-1], filled=True);

In [ ]:
fig.savefig("depth3.svg")

### Huge tree (depth=20)

In [ ]:
model = DecisionTreeRegressor(max_depth=20)

In [ ]:
%%time
model.fit(X_train, y_train)

Train error

In [ ]:
mean_squared_error(
    y_true=y_train,
    y_pred=model.predict(X_train)
)

Test error

In [ ]:
mean_squared_error(
    y_true=y_test,
    y_pred=model.predict(X_test)
)

### Overfitting

Lets see how training and test error changes with `max_depth`

In [ ]:
y_train

In [ ]:
results = []

for depth in range(1, 21):
    model = DecisionTreeRegressor(max_depth=depth)
    model.fit(X_train, y_train)
    
    result = {
        "depth": depth,
        "train_error": mean_squared_error(y_train, model.predict(X_train)),
        "test_error": mean_squared_error(y_test, model.predict(X_test))
    }
    
    results.append(result)

In [ ]:
results_df = pd.DataFrame(results)

In [ ]:
results_df

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.plot(results_df.depth, results_df.train_error, label="train error")
plt.plot(results_df.depth, results_df.test_error, label="test error")
plt.legend()

We can see how, when `max_depth` increases above ~6:
 * training error decresases (more precise on training samples)
 * test error increases (model is memorizing training set and not generalizing very well)

This is the famous overfitting! And this is why **test error** is the one you should look at!

### Other hyperparameters

As well as `max_depth`, there are other **hyperparameters** that let us build different trees' architectures of the family DecisionTreeRegressor:
 * `min_samples_split`: the minimum number of samples required to split an internal node  
 * `max_features`: the number of features to consider when looking for the best split  

In [ ]:
model = DecisionTreeRegressor(max_depth=5, min_samples_split=100, max_features=8, random_state=666)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
print(f"train error: {mean_squared_error(y_train, model.predict(X_train))}")
print(f"test error: {mean_squared_error(y_test, model.predict(X_test))}")

### Grid search

Lets find the **best** combination of hyperparameters, i.e. the ones yielding the least test error, among a prescribed grid of values for each hyperparameter

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
gs = GridSearchCV(
    estimator=DecisionTreeRegressor(),
    param_grid={
        "max_depth": [5, 6],
        "min_samples_split": [50, 100, 300, 1000],
        "max_features": [7, 11]
    },
    cv=5,
    verbose=3,
    scoring="neg_mean_squared_error",
    return_train_score=True
)

It will try 4 * 2 = 16

In [ ]:
%%time
gs.fit(X_train, y_train)

Lets sort all trees by their performance:

In [ ]:
grid_search_results = pd.DataFrame(gs.cv_results_)
# we only keep some of the information
grid_search_results = grid_search_results[['param_max_depth', 'param_max_features', 'param_min_samples_split',
       'mean_test_score', 'mean_train_score']]

In [ ]:
grid_search_results.sort_values("mean_test_score", ascending=False).head(10)

We can access the best estimator of the grid search in this way

In [ ]:
best_tree = gs.best_estimator_

In [ ]:
best_tree

In [ ]:
mean_squared_error(best_tree.predict(X_test), y_test)

## Feature importance

How important were features for predicting `quality`? DecisionTreeRegressor has an attribute `feature_importances_`

In [ ]:
best_tree

In [ ]:
feature_imp = pd.Series(best_tree.feature_importances_, index=df.columns[:-1]).sort_values(ascending=False)

In [ ]:
feature_imp

In [ ]:
sns.barplot(x=feature_imp.values, y=feature_imp.index)

In [ ]:
fig = plt.figure(figsize=(20, 20))
plot_tree(best_tree, feature_names=df.columns[:-1], filled=True);

Save in format `.svg` for non-pixeled image!!

In [ ]:
fig.savefig("decision_tree.svg", facecolor="white")

## Summary

 * Decision trees are useful for regression (`DecisionTreeRegressor`) and classification (`DecisionTreeClassifier`)
 * Their behavior is quite intuitive
 * Their behavior is interpretable and explainable

 * Decision trees overfit when `max_depth` becomes very big (obvious, individual leaves at the end)
 * Prevent overfitting (always, not only in tree based methods) by looking at test error and train error

 * One decision tree is often not a very powerful ML algorithm
 * Decision trees are the building blocks of more advanced and superpowerful algorithms